In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [4]:
tau_d = 0.01
L_s = 0.01
R_s = 0.01
K_p = L_s/tau_d
K_i = R_s/tau_d
params_dict = {'L_s':L_s,'R_s':R_s,
               'N_pp':2.0,'Phi':1.0,
              'K_p':K_p,'K_i':K_i}


u_ini_dict = {'tau_r_ref':0.0,'i_d_ref':0.0,'omega_r':0.0}  # for the initialization problem
u_run_dict = {'tau_r_ref':0.0,'i_d_ref':0.0,'omega_r':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['i_d','i_q','xi_i_d','xi_i_q']    # [inductor current, PI integrator]
y_ini_list = ['tau_r','omega_e','v_d','v_q','i_q_ref'] # for the initialization problem
y_run_list = ['tau_r','omega_e','v_d','v_q','i_q_ref'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [10]:
# pmsm:
di_d = 1/L_s * (  v_d + L_s * omega_e * i_q - R_s * i_d  )
di_q = 1/L_s * (  v_q - L_s * omega_e * i_d - omega_e * Phi - R_s * i_q  )
eq_tau_r = -tau_r + 3/2*Phi*N_pp*i_q
eq_omega_e = -omega_e + N_pp*omega_r

# ctrl:
epsilon_i_d =  i_d_ref - i_d
epsilon_i_q =  i_q_ref - i_q
u_d = K_p*epsilon_i_d + K_i*xi_i_d
u_q = K_p*epsilon_i_q + K_i*xi_i_q

eq_v_d = -u_d + v_d + L_s * omega_e * i_q 
eq_v_q = -u_q + v_q - L_s * omega_e * i_d -  omega_e * Phi 
dxi_i_d = epsilon_i_d
dxi_i_q = epsilon_i_q
eq_i_q_ref = - tau_r_ref + 3/2*Phi*N_pp*i_q_ref


In [12]:
f_list = [di_d,di_q,dxi_i_d,dxi_i_q]
g_list = [eq_tau_r,eq_omega_e,eq_v_d,eq_v_q,eq_i_q_ref]
h_dict = {'u_d':u_d,'u_q':u_q,'tau_r_ref':tau_r_ref,'v_pk':(v_d**2+v_q**2)**0.5}

sys = {'name':f"pmsm_ctrl",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)


0.04986906051635742
0.030918121337890625
Compilation time: 4.82 s


In [7]:
sys['jac_ini']

Matrix([
[    -R_s/L_s,      omega_e, 0, 0,  0,                  i_q, 1/L_s,     0,                0],
[    -omega_e,     -R_s/L_s, 0, 0,  0, (-L_s*i_d - Phi)/L_s,     0, 1/L_s,                0],
[          -1,            0, 0, 0,  0,                    0,     0,     0,                0],
[           0,           -1, 0, 0,  0,                    0,     0,     0,                1],
[           0, 1.5*N_pp*Phi, 0, 0, -1,                    0,     0,     0,                0],
[           0,            0, 0, 0,  0,                   -1,     0,     0,                0],
[           0,  L_s*omega_e, 0, 0,  0,              L_s*i_q,     1,     0,                0],
[-L_s*omega_e,            0, 0, 0,  0,       -L_s*i_d - Phi,     0,     1,                0],
[           0,            0, 0, 0,  0,                    0,     0,     0, 1.5*N_pp*Phi + 1]])